<img src="header.png" width="2048" height="480" align="center"/>

# Preliminary Experiments in Catalog Emulation using Bayesian Neural Networks

Ji Won Park, Phil Marshall (SLAC/Stanford)

Created for the SSim telecon on 6/20/2019

This Note describes initial experiments in emulating Data Release Processing (DRP) Object catalogs using a Bayesian approach to deep learning, called Bayesian neural networks (BNN). We show that BNNs can be applied to estimate both observational and modeling uncertainties.

Contents
==

### Introduction

- Catalog emulation: what and why
- Why use deep learning?
- Bayesian inference with deep learning

### Method
- Training data
- Design choices in the architecture
    - Flexible posterior
    - Better-calibrated uncertainty estimates

### Results

### Conclusions

Introduction
==

Catalog emulation
===============
The stochastic forward problem! Given the truth properties of an object and the observation conditions, we wish to infer the observed properties.

```python
def emulate(extragalactic catalog joined with Truth catalog, OpSim):

    # process approximating the DRP image processing 
    # and object detection/measurement
    
    return emulated DRP Object catalog
```

Why emulate catalogs?
==

Less computational resource!
- Compare 1 million CPU hours for processing the 300 deg$^2$ of DC2 images vs. ~ minutes for deep learning inference

Why emulate with deep learning?
==

- Capture complex patterns in data that elude our physics intuition/understanding
    - e.g. dependencies across quantities

- Schema-specific emulation
    - e.g. simultaneously infer psFlux, KronFlux, and ApertureFlux

- More complex noise modeling
    - Go beyond Gaussian sky noise

Limits of traditional neural networks
==

A common criticism against traditional neural networks (NNs) is that they are black boxes that output a point estimate, with no reference to the predictive uncertainty.

<center><img src=https://imgs.xkcd.com/comics/machine_learning.png width="300" height="600" align="center"/></center>


Casting deep learning as Bayesian models
==

Let $X = \{ x_1, \cdots, x_N \}$ and $Y = \{y_1, \cdots, y_N\}$ be the training dataset.

(For us, $X$ is taken from extragalactic, Truth, and OpSim catalogs and $Y$ from the DRP Object catalog.)


A traditional NN has deterministic weights $W$ parameterizing a mapping from $X$ to $Y$, $f^{W}: X \rightarrow Y$. Instead, a __Bayesian NN__ (BNN) places distributions over $W$. The output $f^{W}(x)$, $x \in X$, is random.

__Goal:__ Evaluate the posterior

$$p(W | X, Y) = \frac{p(Y | X, W) p(W)}{p(Y | X)}$$

so that we can predict on a new point $(x^*, y^*)$:

$$p(y^* | x^*, X, Y) = \int p(y^* | x^*, W) p(W | X, Y) dW $$

This like averaging predictions from an ensemble of NNs weighted by posterior probabilities of their weights.

Ingredients for inference with a BNN
==

1. Sampling distribution over the DRP quantities $p\left(y | f^{W}(x) \right)$, aka "model likelihood"

    e.g. $y | f^{W}(x) \sim N(f^{W}(x), \sigma^2(x)I)$ where $\sigma^2(x)$ is a noise scalar to be learned
    
    More flexibility can be achieved with a non-diagonal covariance matrix $\Sigma$.

Although we'd ideally want to span the whole data space with the DC2 DRP-like catalogs, this is not feasible since there are so many quantities to predict. We want the model to output uncertainty estimates that *depends on the data*.

2. Prior on the weights $p(W)$

    e.g. $W \sim N(0, \tau^2I)$ for some fixed $\tau^2$ (which we specify) is equivalent to $L_2$ regularization

Approximate posterior
==
Evaluating posterior $p(W | X, Y)$ is intractible. $\implies$ Approximate it with the variational distribution $q_\theta(W)$.

## Form of the approximate posterior $q_\theta(W)$

Let's index layers by $l = 1, \cdots, L$ and nodes in layer $l$ by $j=1, \cdots, J_{l}$. Define $M_l \in \mathbb{R}^{J_{l} \times J_{l-1}}$ as the matrix of weights that takes nodes from layer $l-1$ to $l$.

<center><img src=https://miro.medium.com/max/644/1*dEi_IkVB7IpkzZ-6H0Vpsg.png width="200" height="100" align="center"/></center>
(This NN has $L = 2$, with $J_0 = 3$ for the input layer and $J_1 = 4, J_2 = 2$. Note $M_2 \in \mathbb{R}^{4 \times 2}$.)

Choose $q_\theta(W) = \prod_{l=1}^L q_{M_l}(W_l)$ such that

$$W_l = M_l \cdot \textrm{diag}\left( \textrm{Bernoulli}(1-p_l)^{J_{l-1}} \right)$$ 

where $p_l \in \mathbb{R}$ is the dropout probability in layer $l$. We optimize on $\theta=\{M_l\}_{l=1}^L$.

Variational inference
==

Minimize the KL divergence between $q_\theta(W)$ and $p(W | X, Y)$, i.e.

$\textrm{min}_\theta \quad KL(q_\theta(W) | p(W | X, Y))$

$\iff$ maximize the log-evidence lower bound (ELBO):

$\textrm{max}_\theta \quad \int q_\theta(W) \log p(Y | X, W) dW - KL(q_\theta(W) || p(W))$

$\implies$ maximize the approximate ELBO:

$\textrm{max}_\theta \quad \sum_{n=1}^N \log p\left( y_n|  f^W (x_n) \right) - \lambda \sum_{l=1}^L ||W_l||^2 $, where $W \sim q_\theta(W)$

Sampling from the posterior
==

Training determines an optimal $q^*_\theta(W)$ maximizing the ELBO.

We can sample from $q^*_\theta(W)$ by running forward passes with dropout several ($T$) times.

__Monte Carlo dropout__ (Gal 2015): 

For a new test point $x^*$,

$$p(y^* | x^*, X, Y) = \int p(y^* | x^*, W) q_\theta^*(W) dW $$

$$E(y^*) \approx \frac{1}{T}\sum_{t=1}^T f^{W_t} (x^*) \quad \textrm{where } W_t \sim q^*_\theta(W)$$

How to estimate uncertainty of this prediction on $y^*$?

Uncertainty quantification
==

Uncertainty can be categorized into two classes. Say we have a new test point, $(x^*, y^*)$.

1. Aleatoric uncertainty: due to noise inherent in the observations
    - sky noise, blurring due to PSF
    - specify this in the likelihood, e.g. $y | f^{W}(x) \sim N(f^{W}(x), \sigma_{al}^2(x)I)$ 
    - have the network predict it and average across MC samples, i.e. $\sigma^2_{al}(y^*) \approx \frac{1}{T} \sum_{t=1}^T \sigma^2_{al, t}(x^*)$

2. Epistemic uncertainty: modeling uncertainty
    - choice of model structure, parameters for a chosen model (under/overfitting)
    - estimated as variance in prediction across MC samples, i.e. $\sigma^2_{ep}(y^*) \approx Var\left( f^{W_t}(x^*) \right)$
    - Used as a diagnostic for model fit

Whereas epistemic uncertainty can be reduced with more data, aleatoric uncertainty is fixed by the LSST systematics and statistical errors.

Methods
==

Notes on selection
==

For our initial run, we sample objects in Tract 4850. No explicit cut was applied, but many objects were lost due to:

1. Matching (-93%, 1.2M $\rightarrow$ 85K)
    - Truth and DRP catalogs have been matched positionally (closest match within 1").
    - Currently looking into using the output from ImSim to improve the matching quality

2. Cleaning (additional -7%, 85K $\rightarrow$ 80K)
    - Discarded all objects in the Object catalog that had null values in any of the chosen target quantities

### Training data for Object catalog emulation

$X = \{ x_n\}_{n=1}^N$ ($N \sim$ 80K) where each $x_n \in \mathbb{R}^{23}$ consists of
- __Truth (protoDC2) catalog__ quantities: total truth magnitudes `ugrizy` converted into flux units, truth `ra`/`dec`, `redshift`, `not_star` (whether it's extended; 10% were stars), 
`size_true`, `size_minor_true`, `e1`, `e2` for both bulge and disk, and `bulge_to_total_ratio_i`
- __OpSim__ quantities: `fiveSigmaDepth` in flux units, `FWHMeff`, `filtSkyBrightness` in flux units, `airmass`, visit-averaged for this object

$Y = \{ y_n\}_{n=1}^N$ where each $y_n \in \mathbb{R}^{34}$ consists of __Run 1.2i Object catalog__ quantities
- fluxes `cModelFlux`, `psFlux`, `base_CircularApertureFlux_70_0_instFlux`, `ext_photometryKron_KronFlux_instFlux` for all filters
- first moments `x`/`y`, offset of observed `ra`/`dec` from truth, second moments `Ixx`$^{0.5}$/ `Ixy`/`Iyy`$^{0.5}$, `extendedness`, PSF second moments `Ixx`/`Ixy`/`Iyy`

### Training data generation workflow (for later reference!)

All intermediate catalogs are __bolded__. First, decide on tract ID (filter all downloads in below steps to this tract).

1. Associate __Source__ with __OpSim__ : fetch __Source__ $\rightarrow$ join with __OpSim__ on keys `['visit', 'filter']`, `['obsHistID', 'filter]` (in `associate_with_opsim_demo.ipynb`) $\Rightarrow$ __Source+OpSim__

2. Associate __Object__ with __OpSim__ : fetch __Object__ $\rightarrow$ join with __Source+OpSim__ on key `objectID` and group by 'objectID' and average relevant __OpSim__ columns (in `associate_with_opsim_demo.ipynb`) $\Rightarrow$ __Object+OpSim__

3. Obtain truth info for galaxies (in `match_to_truth_demo.ipynb`): fetch __Truth__ $\rightarrow$ join with __protoDC2__  on keys `galaxy_id`, `object_id` $\Rightarrow$ __galaxies_truth__

4. Obtain truth info for stars (in `match_to_truth_demo.ipynb`): query stars in __Truth__ $\Rightarrow$ __stars_truth__

5. Combine __galaxies_truth__ and __stars_truth__ $\Rightarrow$ __X_raw__ (in `match_to_truth_demo.ipynb`)

6. Match __X_raw__ to __Object+OpSim__ positionally $\Rightarrow$ __Y_Object__ (in `match_to_truth_demo.ipynb`)

7. Join __Y_Object__ with __Source+OpSim__ on previously `objectID` $\Rightarrow$ __Y_Source__ (in `match_to_truth.ipynb`)

### Known validation issues in Run 1.2i

- Biased photometry when compared to the Truth catalog, c.f. Reference catalog

# Design choices for an expressive posterior
## 1. Specifying a flexible sampling distribution

We ensure that our approximate posterior is expressive enough to handle banana shapes and outliers.
Recall:
$$p(y^* | x^*, X, Y) = \int p(y^* | x^*, W) q_\theta^*(W) dW$$

For the __sampling distribution__, we use a mixture of two Gaussians with full covariance matrices:

$$p\left(y | f^{W}(x) \right) = \alpha(x) \phi \left(y | \mu_1(x), \Sigma_1(x) \right) 
+ \left(1 - \alpha(x) \right) \phi\left(y | \mu_2(x), \Sigma_2(x) \right) $$
where $\phi(\cdot | \mu, \Sigma)$ is the density of a multivariate normal with mean $\mu$ and covariance matrix $\Sigma$.

The covariance matrices $\Sigma_1$, $\Sigma_2$ encode the aleatoric uncertainty.

## 2. Relaxing the dropout condition

In the regular dropout framework, we need to do a grid search over the dropout probabilities. This often forces us to use one dropout probability $p$ for all the layers.

__Concrete dropout__ (Gal 2017) relaxes the all-or-nothing step function of dropout into a continuous (sigmoid) distribution. This enables us to optimize on the $\{p_l \}_{l=1}^L$, now allowed to differ across layers, and obtain better-calibrated uncertainty estimates.

<center><img src=https://apprize.info/python/scratch/scratch.files/image198.jpg width="400" height="200" align="center"/></center>

Results (on google slides for now)
==

Conclusion
==

- Some of the training examples are corrupted due to faulty matching.
    - More training examples may help reduce the damage.
    - FoF matching may fare better (if so, update the Run 1.2i matching tutorial!)
- More tuning!

Next steps
==

- Scale up to a bigger training dataset (other tracts) using FoF matching in Run 1.2i, and then in Run 2.1i
- Plot pairwise correlations between quantities

Collaborative bibliography
==

I'm writing a paper on lens finding trained on the emulated catalogs, but literature on astronomical catalog emulation is sparse! It'd be great to have a running platform for collaborative bibliography on this topic. Relevant topics are also welcome, including (but not limited to):

- Uncertainty quantification using DL
    - Alternatives using generative adversarial networks (GANs)
- Use cases in astronomy
- Stochastic forward problem

I've started a bibliography here:

Code
==

This presentation is both a DESC Note and a Jupyter notebook, available at my [deep-learning-notebooks repo](https://github.com/jiwoncpark/deep-learning-notebooks).

The code for data generation and training is currently available at my [fast-forward repo](https://github.com/jiwoncpark/fast-forward), but will be merged to the [LSSTDESC/derp repo](https://github.com/LSSTDESC/derp) soon after substantial style improvement via Phil's code review!

Note: some of the code is very hacky (sorry!) but I've commented them in detail...

### Please let me know if you're interested in being my reviewer!

Thank you! :-)
==

Backup
==

About Run 1.2i
==

- Image:
    - Available area: 29.4 deg$^2$
    - Median $5\sigma$-depth $r \sim 25.7$ (~5 years)
- protoDC2 v.3 (Korytov 2018)
    - Area: 25 deg$^2$
    - Redshift range: $0 < z < 1$
    - Based on a semi-analytic model from Galacticus SAM (Benson 2012) on the ANL AlphaQ simulation (256 Mpc/h) (Habib et al 2016), a downscaled version of the Outer Rim simulation
- OpSim
    - `/global/projecta/projectdirs/lsst/groups/SSim/DC2/minion_1016_desc_dithered_v4.db`